---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 2 - Introduction to NLTK

In part 1 of this assignment you will use nltk to explore the Herman Melville novel Moby Dick. Then in part 2 you will create a spelling recommender function that uses nltk to find words similar to the misspelling. 

## Part 1 - Analyzing Moby Dick

In [8]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

import pandas as pd
import numpy as np

# If you would like to work with the raw text you can use 'moby_raw'
with open('moby.txt', 'r') as f:
    moby_raw = f.read()
    
# If you would like to work with the novel in nltk.Text format you can use 'text1'
moby_tokens = nltk.word_tokenize(moby_raw)
text1 = nltk.Text(moby_tokens)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Example 1

How many tokens (words and punctuation symbols) are in text1?

*This function should return an integer.*

In [3]:
def example_one():
    
    return len(nltk.word_tokenize(moby_raw)) # or alternatively len(text1)

example_one()

254989

### Example 2

How many unique tokens (unique words and punctuation) does text1 have?

*This function should return an integer.*

In [4]:
def example_two():
    
    return len(set(nltk.word_tokenize(moby_raw))) # or alternatively len(set(text1))

example_two()

20755

### Example 3

After lemmatizing the verbs, how many unique tokens does text1 have?

*This function should return an integer.*

In [5]:
from nltk.stem import WordNetLemmatizer

def example_three():

    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w,'v') for w in text1]

    return len(set(lemmatized))

example_three()

16900

### Question 1

What is the lexical diversity of the given text input? (i.e. ratio of unique tokens to the total number of tokens)

*This function should return a float.*

In [6]:
def answer_one():
    
    total_tokens = len(nltk.word_tokenize(moby_raw))
    unique_tokens = len(set(nltk.word_tokenize(moby_raw)))
    
    return unique_tokens/total_tokens

answer_one()

0.08139566804842562

### Question 2

What percentage of tokens is 'whale'or 'Whale'?

*This function should return a float.*

In [42]:
def answer_two():
    
    from nltk.probability import FreqDist
    dist = FreqDist(text1)
    whale = dist['whale'] + dist['Whale']
    
    return (whale/len(text1))*100

answer_two()

0.4125668166077752

### Question 3

What are the 20 most frequently occurring (unique) tokens in the text? What is their frequency?

*This function should return a list of 20 tuples where each tuple is of the form `(token, frequency)`. The list should be sorted in descending order of frequency.*

In [ ]:
def answer_three():
    
    from nltk.probability import FreqDist
    dist = FreqDist(text1)
    
    d = [dist[i] for i in dist.keys()]
    x = pd.DataFrame(data = d, index=dist.keys(), columns=['Freq']).sort_values(by=['Freq'], ascending= False)
    
    result = [(x.index[i], x.values[i][0]) for i in range(20)]
    
    return result

answer_three()

### Question 4

What tokens have a length of greater than 5 and frequency of more than 150?

*This function should return an alphabetically sorted list of the tokens that match the above constraints. To sort your list, use `sorted()`*

In [ ]:
def answer_four():
    
    from nltk.probability import FreqDist
    dist = FreqDist(text1)
    
    freqwords = sorted([i for i in dist.keys() if len(i)>5 and dist[i]>150])
    
    return freqwords

answer_four()

### Question 5

Find the longest word in text1 and that word's length.

*This function should return a tuple `(longest_word, length)`.*

In [ ]:
def answer_five():
    
    from nltk.probability import FreqDist
    dist = FreqDist(text1)
    
    x = ''
    for key in dist.keys():
        if len(key) > len(x):
            x = key
    
    return (x, len(x))

answer_five()

### Question 6

What unique words have a frequency of more than 2000? What is their frequency?

"Hint:  you may want to use `isalpha()` to check if the token is a word and not punctuation."

*This function should return a list of tuples of the form `(frequency, word)` sorted in descending order of frequency.*

In [ ]:
def answer_six():
    
    from nltk.probability import FreqDist
    dist = FreqDist(text1)
    
    result = sorted([(dist[i],i)  for i in dist.keys() if i.isalpha()==True and dist[i]>2000], reverse=True)
    
    return result

answer_six()

### Question 7

What is the average number of tokens per sentence?

*This function should return a float.*

In [ ]:
def answer_seven():
    
    from nltk.tokenize import sent_tokenize
    sents = sent_tokenize(moby_raw)
    
    x = 0
    for sent in sents:
         x += len(nltk.word_tokenize(sent))
    
    return x/len(sents)

answer_seven()

### Question 8

What are the 5 most frequent parts of speech in this text? What is their frequency?

*This function should return a list of tuples of the form `(part_of_speech, frequency)` sorted in descending order of frequency.*

In [17]:
def answer_eight():
    
    tags = [tag[1] for tag in nltk.pos_tag(text1)]
    value_counts = pd.Series(data=tags).value_counts()
    
    result = [(value_counts.index[i], value_counts.values[i]) for i in range(5)]
    
    return result

answer_eight()

[('NN', 32730), ('IN', 28657), ('DT', 25867), (',', 19204), ('JJ', 17620)]

## Part 2 - Spelling Recommender

For this part of the assignment you will create three different spelling recommenders, that each take a list of misspelled words and recommends a correctly spelled word for every word in the list.

For every misspelled word, the recommender should find find the word in `correct_spellings` that has the shortest distance*, and starts with the same letter as the misspelled word, and return that word as a recommendation.

*Each of the three different recommenders will use a different distance measure (outlined below).

Each of the recommenders should provide recommendations for the three default words provided: `['cormulent', 'incendenece', 'validrate']`.

In [19]:
from nltk.corpus import words
#import nltk
#nltk.download('words')

correct_spellings = words.words()

[nltk_data] Downloading package words to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


### Question 9

For this recommender, your function should provide recommendations for the three default words provided above using the following distance metric:

**[Jaccard distance](https://en.wikipedia.org/wiki/Jaccard_index) on the trigrams of the two words.**

*This function should return a list of length three:
`['cormulent_reccomendation', 'incendenece_reccomendation', 'validrate_reccomendation']`.*

In [40]:
def answer_nine(entries=['cormulent', 'incendenece', 'validrate']):
    #from nltk.util import ngrams
    
    result = []
    for entry in entries:
        
        #Pick from candidates with same starting letter
        same_start_letter = [word for word in correct_spellings if word[0] == entry[0]]
        trigrams = set(nltk.ngrams(entry, n=3))
        
        x = 1
        for word in same_start_letter:
            jd = nltk.jaccard_distance(trigrams, set(nltk.ngrams(word,n=3)))
            if jd < x:
                x = jd
                correction = word
                
        result.append(correction)
    
    return result

    
answer_nine()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: generator 'ngrams' raised StopIteration
  del sys.path[0]


['corpulent', 'indecence', 'validate']

### Question 10

For this recommender, your function should provide recommendations for the three default words provided above using the following distance metric:

**[Jaccard distance](https://en.wikipedia.org/wiki/Jaccard_index) on the 4-grams of the two words.**

*This function should return a list of length three:
`['cormulent_reccomendation', 'incendenece_reccomendation', 'validrate_reccomendation']`.*

In [41]:
def answer_ten(entries=['cormulent', 'incendenece', 'validrate']):
    
    #from nltk.util import ngrams
    
    result = []
    for entry in entries:
        
        #Pick from candidates with same starting letter
        same_start_letter = [word for word in correct_spellings if word[0] == entry[0]]
        fourgrams = set(nltk.ngrams(entry, n=4))
        
        x = 1
        for word in same_start_letter:
            jd = nltk.jaccard_distance(fourgrams, set(nltk.ngrams(word,n=4)))
            if jd < x:
                x = jd
                correction = word
                
        result.append(correction)
    
    return result
    
answer_ten()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'ngrams' raised StopIteration
  


['cormus', 'incendiary', 'valid']

### Question 11

For this recommender, your function should provide recommendations for the three default words provided above using the following distance metric:

**[Edit distance on the two words with transpositions.](https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance)**

*This function should return a list of length three:
`['cormulent_reccomendation', 'incendenece_reccomendation', 'validrate_reccomendation']`.*

In [37]:
def answer_eleven(entries=['cormulent', 'incendenece', 'validrate']):
    
    result = []
    for entry in entries:
        
        #Pick from candidates with same starting letter
        same_start_letter = [word for word in correct_spellings if word[0] == entry[0]]
        
        x = 9999
        for word in same_start_letter:
            jd = nltk.edit_distance(entry, word, transpositions=True)
            if jd < x:
                x = jd
                correction = word
                
        result.append(correction)
    
    return result
    
answer_eleven()

['corpulent', 'intendence', 'validate']